In [1]:
type Size =
    | Large
    | Medium
    | Small
    
type Lives =
    | Three
    | Two
    | One

let nextLives (life: Lives) =
    match life with
    | Three -> Two
    | Two -> One
    
type Ship = {
        Pos: float*float
        Vel: float*float
        Ang: float
        Lives: Lives
        Size: float    // el radio
    }

type Asteroid = {
    Pos: float*float
    Vel: float*float
    Size: Size
    }

type Bullet = {
    Pos: float*float
    Ang: float
    Range: float
}

type Dir =
    | Left
    | Right

type Saucer = {
    Pos: float*float
    Dir: Dir
    Size: float
}

type Rotation =
    | Positive
    | Negative
    | Zero

type Input = {
    Thrust: bool
    Shoot: bool
    Rot: Rotation
}

type Game = {
    Ship: Ship
    Asteroids: Asteroid list
    Bullets: Bullet list
    Saucer: Saucer Option
    Score: int
    Lives: Lives
}

type Gamestate =
    | Playing of Game
    | Gameover

let fps = 24
let aspect_ratio = 1.5
let rand = Random()   // inicializo instancia de Random()
let maxAsteroidVel = 20.

In [2]:
let sizeMap (size: Size) = 
    match size with
    | Large -> 0.04
    | Medium -> 0.02
    | Small -> 0.01

let min_abs x y = 
    min (abs x) (abs y)

let distance (pos1:float*float) (pos2:float*float) = 
    let dx1 = fst pos1 - fst pos2
    let dy1 = snd pos1 - snd pos2

    let dx2 = min_abs dx1 (aspect_ratio - dx1)
    let dy2 = min_abs dy1 (1. - dy1)
    Math.Sqrt(dx2 ** 2 + dy2 ** 2) 

let cartesianToPolar (vec: float*float) =
    let (x, y) = vec                               
    let r = Math.Sqrt(x**2 + y**2)
    let theta = Math.PI/2.0 - Math.Atan2 (y, x) // REVISAR!
    r, theta

let polarToCartesian (r: float) (theta: float) =
    let x = r * Math.Cos(theta)
    let y = r * Math.Sin(theta)
    (x, y)

let checkCollisionShipAsteroid (ship: Ship) (asteroid: Asteroid) = 
    // Devuelve verdadero si hay superposición entre la nave y un asteroide
    let shipAstDistance = distance ship.Pos asteroid.Pos
    let radiiSum = sizeMap asteroid.Size + ship.Size
    shipAstDistance < radiiSum

let checkCollisionShipBullet (ship: Ship) (bullet: Bullet) =
    // Devuelve verdadero si hay superposición entre la nave y una bala
    let shipBulletDistance = distance bullet.Pos ship.Pos
    shipBulletDistance < ship.Size //la bala se supone puntual

let checkCollisionBulletAsteroid (bullet: Bullet) (asteroid: Asteroid) =
    let bulletAsteroidDistance = distance bullet.Pos asteroid.Pos
    bulletAsteroidDistance < sizeMap asteroid.Size 

let isBullDestroyed (bull: Bullet) (asts: Asteroid List) =
    // true cuando bull es destruída por contacto con algún asteroide
    let collisions =
        asts
        |> List.filter (fun x -> checkCollisionBulletAsteroid bull x)
    not (collisions = [])

let isAstDestroyed (bulls: Bullet List) (ast: Asteroid) =
    // true cuando asteroide es destruido por contacto con alguna bala
    let collisions =
        bulls
        |> List.filter (fun x -> checkCollisionBulletAsteroid x ast)
    not (collisions = [])

let astSplit (ast: Asteroid) = 
    let spawn2RandAsts (pos: float*float) (size: Size) =

        let randVel () : float*float=
            let r = rand.NextDouble() * maxAsteroidVel
            let ang = float (rand.Next(16)) * Math.PI/8.
            polarToCartesian r ang

        let vel1 = randVel ()
        let vel2 = randVel ()
        let randAsts : Asteroid List = [{Pos = pos; Vel = vel1; Size = size};
                                        {Pos = pos; Vel = vel2; Size = size}]
        randAsts

    match ast.Size with
    | Large -> spawn2RandAsts ast.Pos Medium
    | Medium -> spawn2RandAsts ast.Pos Small 
    | Small -> []

In [3]:
let bulletsVelocity = 0.1
let maxBullRange = 20

let trueModulo (x: float) (modulo: float) =
    // Esta función hace que los objetos que traspasan un borde la pantalla regresen por otro
    let y = x % modulo
    match y with
    | _ when y < 0.0 -> y + modulo
    | _ -> y

let moveBullet (bullet: Bullet) = 
    let Vx = -bulletsVelocity*Math.Cos(bullet.Ang)
    let Vy = -bulletsVelocity*Math.Sin(bullet.Ang)

    let newPosX = trueModulo ((fst bullet.Pos) + Vx/(float fps)) (aspect_ratio)
    let newPosY = trueModulo ((snd bullet.Pos) + Vy/(float fps)) (1.0)
    let newPosition = (newPosX, newPosY)
    
    let newRange = bullet.Range + bulletsVelocity/(float fps)

    let newBullet = 
        { bullet with
            Pos = newPosition
            Range = newRange
        }
    newBullet

let moveAndClearBullets (bullets: List<Bullet>) = 
    let maxBullRange = 50
    bullets
    |> List.map (fun b -> moveBullet b) // Actualiza la posición de las balas y contabiliza su recorrido
    |> List.filter (fun b -> b.Range <= maxBullRange) // Elimina las balas cuyo recorrido supera el máximo establecido


In [4]:
// ------------------------------------
// creo que en el juego cuando los objetos se van de la pantalla, aparecen partidos en los dos extremos. 
// CHEQUEAR
// ------------------------------------

let shiftGeneral (pos: float*float) (vel: float*float) =
    let newPosX = trueModulo (fst pos + fst vel) aspect_ratio
    let newPosY = trueModulo (snd pos + snd vel) 1.0
    let newPosition = (newPosX, newPosY)
    newPosition

let newPosAsteroid (asteroid: Asteroid) =
    // Función que actualiza la posición de los asteroides
    // A los módulos se les adiciona el tamaño del objeto para que la reaparicion ocurra cuando el objeto deja de visualizarse en la pantalla
    shiftGeneral asteroid.Pos asteroid.Vel

let newPosShip (ship: Ship) =
    shiftGeneral ship.Pos ship.Vel

In [5]:
let startGame (game: Game) =
    // Reestablece el estado de la nave en el juego
    let newShip =
        { game.Ship with
            Pos = (0.5, aspect_ratio/2.0)
            Ang = 0.0
            Vel = (0.0, 0.0)
        }
    let newGame = Playing {   
                            Ship = newShip
                            Asteroids = []
                            Bullets = []
                            Saucer = None
                            Score = 0
                            Lives = Three
                            }
    newGame


In [6]:
let shootBullet (ship: Ship) (bullets: List<Bullet>) (input: Input) = 
    // Agrega una bala a la lista de balas cuando se ejecuta la acción de disparar
    let fire (ship: Ship) (bullets: List<Bullet>) =
        let newBullet = 
            {
                Pos = (fst ship.Pos + ship.Size*Math.Cos(ship.Ang),
                       snd ship.Pos + ship.Size*Math.Sin(ship.Ang))
                Ang = ship.Ang
                Range = 0.0
            }
        bullets @ [newBullet]
    
    match input.Shoot with
    | true -> fire ship bullets
    | false -> bullets


In [7]:
let deltaAngle (input: Input) = 
    // Modifica la orientación de la nave en pi/8 si el input así lo indica
    let angQuantum = Math.PI/8.0
    let newAngle (rotation: Rotation) =
        match rotation with
        | Positive -> angQuantum
        | Negative -> -angQuantum
        | Zero -> 0.0
    newAngle input.Rot

In [8]:
// Funciones no finalizadas!
let renormalizeVelocity (vel: float*float) = 
    // Esta función auxiliar sirve para acelerar y cambiar la dirección conservando la velocidad
    let (r, theta) = cartesianToPolar vel
    let norm = Math.Sqrt ((fst vel)**2 + (snd vel)**2)
    let (vx, vy) = vel
    (vx * r / norm, vy * r / norm)

let tupleAdd (tup1: float*float) (tup2: float*float) =
    (fst tup1 + fst tup2, snd tup1 + snd tup2)

let accelerateShip (ship: Ship) (input: Input) =
    let shipMaxVel = 1                      // /(float fps) ?
    let shipAcc = 1.0/(float fps)
    let shipDesacc = 0.5/(float fps)  // desacelera más lento
    let shipMaxVel = 20./(float fps)
    
    let velocityDelta (ship: Ship) (thrust: bool) =
        let (velR, velTheta) = cartesianToPolar ship.Vel
        match thrust with
        | true -> (shipAcc * Math.Cos(ship.Ang), shipAcc * Math.Sin(ship.Ang))  // Aceleración con orientación al ángulo actual
        | false -> (-shipDesacc * Math.Cos(velTheta), -shipDesacc * Math.Sin(velTheta)) // Desaceleración con orientación a la velocidad actual
    
    let velDelta = velocityDelta ship input.Thrust
    let (velR, velTheta) = cartesianToPolar velDelta

    let newVelocity =
        let velFinalUnnorm = tupleAdd ship.Vel velDelta
        match input.Thrust with
        | true ->   match cartesianToPolar velFinalUnnorm with
                    | (r, _) when r <= shipMaxVel -> velFinalUnnorm
                    | (r, _) when r > shipMaxVel  -> velFinalUnnorm |> renormalizeVelocity
        | false ->
                    match cartesianToPolar velDelta with
                    | (r, _) when r = 0  -> (0.0, 0.0)
                    | (r, _) when r <= shipMaxVel -> velFinalUnnorm
    newVelocity

In [9]:
let updateShip (game: Game) (input: Input) = 
    let ship = game.Ship
    let asts = game.Asteroids
    let bulls = game.Bullets

    let astShipColls =
        asts
        |> List.filter (fun x -> checkCollisionShipAsteroid ship x)
    let bullShipColls =
        bulls
        |> List.filter (fun x -> checkCollisionShipBullet ship x)

    let aliveStatus = (astShipColls = [] && bullShipColls = [])
    let newVel = accelerateShip ship input
    let newPos = newPosShip ship
    let newAngle = trueModulo (ship.Ang + deltaAngle input) (2.*Math.PI)
    
    match aliveStatus with
    | true ->   let newShip = {ship with Pos = newPos; Vel = newVel; Ang = newAngle}
                Playing {game with
                            Ship = newShip}

    | false ->  match game.Lives with
                | One -> Gameover
                | _ -> Playing {game with Lives = nextLives ship.Lives}

let updateBullets (game: Game) (input: Input) =
    let ship = game.Ship
    let astList = game.Asteroids
    let bullList = game.Bullets
    let newBullets =
        shootBullet ship bullList input         // las balas se crean sólo por el jugador
        |> List.filter (fun x -> isBullDestroyed x astList = false)         // destrucción de bala por contacto con asteroide
        |> List.filter (fun x -> checkCollisionShipBullet ship x)      // destrucción de bala por contacto con ship
        |> moveAndClearBullets      // se mueven las balas remanentes y se quitan las que alcanzaron el máximo rango
    {game with
        Bullets = newBullets}

let updateAsteroids (game: Game) =
    let ship = game.Ship
    let astList0 = game.Asteroids
    let bullList = game.Bullets

    // separo entre los asteroides destruidos y los que siguen
    let astDeadList, astAliveList =
        astList0
        |> List.partition (fun x -> isAstDestroyed bullList x )

    // a los destruidos se les aplica astSplit, que los elimina si son
    // de size = small, y los separa en dos más pequeños en caso contrario
    let astSplitList =
        astDeadList
        |> List.collect (fun x -> astSplit x)
    
    let astList1 = astAliveList @ astSplitList
    let newAsts =
        astList1
        |> List.map (fun x -> {x with Pos = newPosAsteroid x})
    {game with
        Asteroids = newAsts}

In [10]:
let checkLevelFinished (game: Game) =
    // chequea si el nivel está terminado 
    let ship = game.Ship
    let asts = game.Asteroids

    (asts = [])

let checkGameOver (gamestate: Gamestate) = 
    // chequea game over
    match gamestate with
    | Gameover -> true
    | Playing _ -> false

In [12]:
let spawnAsteroids (game: Game) (num: int) = 
    // spawn de asteroides al inicio del nivel
    // num: cantidad de asteroides a spawnear
    let randAst ()=
        // genera un asteroide Large random ubicado en los bordes de la pantalla
        let x = trueModulo ((rand.NextDouble() - 0.5)*aspect_ratio/2.) aspect_ratio
        let y = trueModulo ((rand.NextDouble() - 0.5)*0.5)  1.0
        let theta = float (rand.Next(16)) * Math.PI/8.
        let vel = polarToCartesian (maxAsteroidVel*0.5) theta 
        {Pos = (x, y); Vel = vel; Size = Large}

    let newAsts = List.init num (fun x -> randAst())
    {game with
        Asteroids = game.Asteroids @ newAsts}

In [ ]:
let spawnSaucer (game: Game) = 
    let saucerSize = 0.1

    let randInBand fraction = 
        // genera número random en una franja de ancho "fracion" centrada en 0.5
        (rand.NextDouble() - 0.5) * fraction + 0.5

    let saucer0 = Some {Pos = (0.0, randInBand 0.9); 
                        Dir = Right;
                        Size = saucerSize}

    let newSaucer = 
        match rand.Next(2) with
        | 0 -> { saucer0 with Dir = Right }
        | 1 -> { saucer0 with Dir = Left }
    {game with
        Saucer = newSaucer}

let moveSaucer (saucer: Saucer) = 
    let saucerVel = 1.0/(float fps)
    let vel = 
        match saucer.Dir with
        | Right -> (saucerVel, 0)
        | Left -> (-saucerVel, 0)
    let newPos = shiftGeneral saucer.Pos vel
    newPos

let saucerShoot (saucer: Saucer) (bullets: list<Bullet>)=
    let saucerFire (saucer: Saucer) (bullets: list<Bullet>) =
        let angRandom = 2.0*Math.PI*rand.NextDouble()
        let newBullet = 
            {
                Pos = (fst saucer.Pos + saucer.Size*Math.Cos(angRandom),
                    snd saucer.Pos + saucer.Size*Math.Sin(angRandom))
                Ang = angRandom
                Life = 0.0
            }
        bullets @ [newBullet]
        
    let fireProbability = 0.3
    match rand.NextDouble() < fireProbability with
    | false -> bullets
    | true -> saucerFire saucer bullets

In [ ]:
let updateSaucer (game: Game) =
    let saucer = game.Saucer

    let bullSaucerColls =
        game.Bullets
        |> List.filter (fun x -> checkCollisionSaucerBullet saucer x)

    let aliveStatus = (bullSaucerColls = [])
    let newPos = moveSaucer saucer
    match aliveStatus with
    | true ->   {saucer with
                    Pos = newPos
                    }
    | false ->  {saucer with
                    Alive = false
                    }